In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

# Loading Source Data (From R Drive)

In [2]:
path = r'R:/DPOE/LEHD LODES/7.5/Source/OD_Data/Unzipped_RD'

In [3]:
# 2018 source data
csv_2018 = glob.glob(path + "/*2018.csv")
list_2018 = []

for filename in csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2018.append(df)

In [4]:
od_2018 = pd.concat(list_2018, axis=0, ignore_index=True)

In [6]:
import sys
sys.getsizeof(od_2018)

5914258000

# Loading The Staging Table (SQL Server)

In [7]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=socioec_data;'
                      'Trusted_Connection=yes;')

#2018
od_db18 ="SELECT * "\
                   "FROM [socioec_data].[lehd_lodes].[od_7_5]"\
                    "WHERE yr= 2018"
db18 = pd.read_sql_query(od_db18, conn)

In [8]:
sys.getsizeof(db18)

1665949506

# Cleaning Source Data

In [ ]:
od_2018 = od_2018.rename({'w_geocode': 'w_geoid', 'h_geocode': 'h_geoid'}, axis=1)

In [ ]:
od_2018 = od_2018.drop(['createdate',], axis=1)

# Cleaning Staging Data

In [ ]:
db18 = db18.drop(['type','yr'], axis=1)

In [ ]:
db18['w_geoid'] = [i.lstrip('0') for i in db18['w_geoid']]

In [ ]:
db18['h_geoid'] = [i.lstrip('0') for i in db18['h_geoid']]

# Filtering for just SD Data (Source Data)

In [ ]:
string_ints = [str(int) for int in od_2018['w_geoid']]

tf = []
for i in string_ints:
    if i[0:4] == '6073':
        tf.append(True)
    else:
        tf.append(False)
        
f_od_2018 = od_2018[tf]

# Filtering for just SD Data (Staging Data)

In [ ]:
string_ints = [str(int) for int in db18['w_geoid']]

tf = []
for i in string_ints:
    if i[0:4] == '6073':
        tf.append(True)
    else:
        tf.append(False)
        
f_db18 = db18[tf]

# Comparison 

In [ ]:
f_db18.columns == f_od_2018.columns

In [ ]:
f_od_2018.dtypes

In [ ]:
f_db18.dtypes

# Checking the shape of the data

In [ ]:
print(f_od_2018.shape)
print(f_db18.shape)

In [ ]:
f_od_2018.shape[0] - f_db18.shape[0]

# Checking Length

In [ ]:
print(len(f_od_2018))
print(len(f_db18))

# To Numeric and Filter by Geoid

In [ ]:
f_od_2018 = f_od_2018.apply(pd.to_numeric)
f_od_2018 = f_od_2018.sort_values(by=['w_geoid'])

In [ ]:
f_db18 = f_db18.apply(pd.to_numeric)
f_db18 = f_db18.sort_values(by=['w_geoid'])

# Count the number of unique IDs

In [ ]:
print(f_od_2018['w_geoid'].nunique())
print(f_db18['w_geoid'].nunique())

# Check if stats are the same

In [ ]:
f_od_2018.describe() == f_db18.describe()

# Round std to make sure they are identical

In [ ]:
# Since this prints out false we see that the rounded version of the standard deviations are all the same
for i in f_od_2018.describe().loc['std',:].round() == f_db18.describe().loc['std',:].round():
    if i == False:
        print(False)